#DATA CORE CURATION

calling the previous notebooks here!

In [0]:
%run /Workspace/Users/dhivyadharshini298@gmail.com/databricks-code-repository/Bread_and_Butter_Transformation_Analytical_operations_DSL/3.DATA_CUSTOMIZATION

##1.Select, Filter

In [0]:
#select
selectdf=custom_df.select("custid","age",upper("prof").alias("profession"))#DSL
selectdf.show(1)
selectdf=custom_df.selectExpr("custid","age","upper(prof) as profession")#SQL
selectdf.show(1)

In [0]:
#filter
filterdf=selectdf.where((col("age")>30) | (col("age")<60))#DSL Syntax
filterdf.show(1)
filterdf=selectdf.where("age>30 or age<60")#SQL Syntax
filterdf.show(1)

In [0]:
selectfilterdf=rawdf1.selectExpr("id","age","upper(profession) as profession").where("age>30 or age<60")
selectfilterdf.show(1)
selectfilterdf.explain()
#or 
selectfilterdf=rawdf1.where("age>30 or age<60").selectExpr("id","age","upper(profession) as profession")
selectfilterdf.show(1)
selectfilterdf.explain()

##2. Derive flags & Columns

In [0]:
#SQL syntax
'''def agecat(agecol):
    if agecol<11:#when(condition,return something)
        return "Children"
    elif agecol>=11 and agecol<=19:
        return "Teens"
    elif agecol>=20 and agecol<=50:
        return "Middleage"
    else:
        return "Oldage"
'''
#DSL syntax
derived_df1=enrich_df7.withColumn("agecat",
                                 when((col("age")<11),lit("Children")).
                                 when(((col("age")>=11) & (col("age")<20)),lit("Teens")).
                                 when(((col("age")>=20) & (col("age")<=50)),lit("Middleage")).
                                 otherwise(lit("Oldage")))
display(derived_df1.limit(10))

In [0]:
#SQL syntax
enrich_df7.createOrReplaceTempView("enrichview")
drive_flag=spark.sql("""select *,
          case when proffirst=proflast then upper(substr(proffirst,1,2)) 
          else upper(concat(substring(proffirst,1,1),substr(proflast,1,1)))
          end as profind 
          from enrichview""")
display(drive_flag.limit(10))

#DSL syntax
derived_df2=derived_df1.withColumn("profind",when(((col("proffirst"))==(col("proflast"))),upper(substring(col("proffirst"),1,2))).otherwise(upper(concat(substring("proffirst",1,1),substring(col("proflast"),1,1)))))
derived_df2.show(20)


##3.Format (deriving columns)

In [0]:

formatted_derived_df=derived_df2.select("*",date_format(col("datadt"),'yyyyMM').alias('datadtyrmth')
                                         ,year("datadt").alias("datadtyr")
                                         ,month("datadt").alias("datadtmth")
                                         ,date_diff("loaddt","datadt").alias("delayindays")
                                         ,last_day("datadt").alias("datadtlastday")
                                         ,date_add("loaddt",1).alias("nextday"))
formatted_derived_df.show(2)

display(formatted_derived_df.limit(20))
     

##4. Group & Aggregate

In [0]:
grouped_df=formatted_derived_df.groupBy("profind")#It will just group
display(grouped_df)

In [0]:
#Calculate profession and agecategory wise count of customers
grouped_agg_df=formatted_derived_df.groupBy("prof","agecat").count()
grouped_agg_df.show(2)
display(grouped_agg_df)

In [0]:
#Not preferred (only for single column) Calculate profession and agecategory wise avg age of customers
grouped_agg_df=formatted_derived_df.groupBy("prof").avg("age").withColumnRenamed("avg(age)","avgage")
grouped_agg_df.show(2)
display(grouped_agg_df)

In [0]:
#More preferred to use agg(aggregate functions) Calculate profession and agecategory wise avg age of customers
grouped_agg_df=formatted_derived_df.groupBy("prof").agg(avg("age").alias("avgage"),min("age").alias("minage"),max("age").alias("maxage"))
grouped_agg_df.show(10)
grouped_agg_df=formatted_derived_df.groupBy("prof","agecat").agg(avg("age").alias("avgage"),min("age").alias("minage"),max("age").alias("maxage"))
grouped_agg_df.show(10)#We will use this grouped aggregated data later post wrangling operation (last stage)
     

##5. Ordering

In [0]:
grouped_agg_df.orderBy("prof","agecat").show(20)#by default ascending order of sorting
grouped_agg_df.orderBy(["prof","agecat"],ascending=[True,False]).show(20)#by default ascending,descending order of sorting
#We will use this grouped aggregated sorted data later post wrangling operation (last stage)

##6. Limit

Difference between limit, take and show/display:

- limit(transformation) is used to limit the number of rows in the dataframe (can be placed anywhere)<br>
- take (action) is used to limit the number of rows in the dataframe and return the result as a python list of rows (can be placed in the last)<br>
- show is used to display the dataframe in a tabular format<br>
- display (databricks specific) is used to display the dataframe in a formatted/advanced tabular format where we can download/visualize etc.,

In [0]:
grouped_agg_df.orderBy(["prof","agecat"],ascending=[True,False]).limit(10).show(20)
grouped_agg_df.orderBy(["prof","agecat"],ascending=[True,False]).take(10)
display(grouped_agg_df.orderBy(["prof","agecat"],ascending=[True,False]).take(10))